In [ ]:
!pip install transformers sentencepiece

In [ ]:
# To scrape Wikipedia
from bs4 import BeautifulSoup
# To access contents from URLs
import requests
# Importar el modelo
import transformers
from transformers import pipeline
import sentencepiece

import re
import torch

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
def procesamiento_tema(tema): 

  # Tema del ChatBot
  tema = tema.strip().replace(' ', '_')

  # Link de Wikipedia
  link = 'https://en.wikipedia.org/wiki/'+ tema

  # Información del link de wikipedia
  url_contenido= requests.get(link).content
  soup_tema = BeautifulSoup(url_contenido, 'html.parser')

  # Encontrar solo el texto de wikipedia
  p_texto = soup_tema.findAll('p')

  # Concatenar en texto
  texto = ''

  for t in p_texto:

    rm_style = t.findAll('style')

    for m in rm_style:
      m.decompose()

    t = t.stripped_strings
    t = ' '.join(list(t))
    t = t.replace(r'( ','(').replace(' )', ')').replace(' .', '.').replace(' ,', ',')
    t = re.sub(u'\[.*\u200b', '', t)

    texto = texto + ' '+ t

  # últimos reemplazos en texto

  replace = [(r'[;»«"]', ''),
            (r' \[\d{1,}\]', ''), 
            (r' \[[a-z]{1,}\]', ''),  
            (r'\s+',' '), 
            (r'\(GMT.*\)', ''),
            (r'^\s', '')]

  for old, new in replace:
      texto = re.sub(old, new, texto) 

  return texto

In [ ]:
device = [0 if torch.cuda.is_available() else 'cpu'][0]
device

In [ ]:
model="anshoomehra/t5-v1-base-s-q-c-multi-task-qgen-v2"

tokenizer= AutoTokenizer.from_pretrained(model)
modelo = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

In [ ]:
def _generate(question, context, device, modelo, modelo_tokenizer):    
    
    input_text = "question: " + question+ "</s> question_context: " + context
    
    input_tokenized = modelo_tokenizer.encode(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=1024).to(device)
    
    summary_ids = modelo.generate(input_tokenized, 
                                       max_length=100, 
                                       min_length=5, 
                                       num_beams=4,
                                       early_stopping=True)
    
    output = [modelo_tokenizer.decode(id, clean_up_tokenization_spaces=True, skip_special_tokens=True) for id in summary_ids] 
    
    return str(output[0])

In [ ]:
texto = procesamiento_tema('2021 Formula One World Championship')
texto

In [ ]:
_generate("Who won the Formula One Driver's Championship?", texto, device, modelo, modelo_tokenizer)   

In [ ]:
_generate("Which team won the Constructor's Championship?", texto, device, modelo, modelo_tokenizer)   

In [ ]:
_generate("Did Mercedes win?", texto, device, modelo, modelo_tokenizer)   

In [ ]:
_generate("What was the budget cap?", texto, device, modelo, modelo_tokenizer)   

In [ ]:
_generate("Who is the race director?", texto, device, modelo, modelo_tokenizer)   

In [ ]:
_generate("Who driver took pole position in the Saudi Arabian Grand Prix?", texto, device, modelo, modelo_tokenizer)   


In [ ]:
_generate("What was the last Grand Prix of the season?", texto, device, modelo, modelo_tokenizer)   


In [ ]:
_generate("how many races were there in the season?", texto, device, modelo, modelo_tokenizer)   

In [ ]:
_generate("how many grand prixes were in 2021?", texto, device, modelo, modelo_tokenizer)   

### The office

In [ ]:
texto = procesamiento_tema('The Office (American season 1)')
texto

In [ ]:
_generate("Who is the main character of the show?", texto, device, modelo, modelo_tokenizer)   

In [ ]:
_generate("When was the show premiere?", texto, device, modelo, modelo_tokenizer)   

In [ ]:
_generate("What country is the series from?", texto, device, modelo, modelo_tokenizer)

In [ ]:
_generate("what is the name of the first episode??", texto, device, modelo, modelo_tokenizer)

In [ ]:
_generate("who is Jim Halpert?", texto, device, modelo, modelo_tokenizer)

In [ ]:
_generate("What days did the episodes air?", texto, device, modelo, modelo_tokenizer)

In [ ]:
_generate("who are the writers of the series?", texto, device, modelo, modelo_tokenizer)

In [ ]:
_generate("How many episodes does the season 1 have?", texto, device, modelo, modelo_tokenizer)

In [ ]:
_generate("What is the Jim Halpert's job?", texto, device, modelo, modelo_tokenizer)

In [ ]:
_generate("What does Pam Beesly?", texto, device, modelo, modelo_tokenizer)

In [ ]:
# https://huggingface.co/consciousAI/question-answering-generative-t5-v1-base-s-q-c
# https://huggingface.co/course/chapter1/9?fw=tf
# https://huggingface.co/MaRiOrOsSi/t5-base-finetuned-question-answering?text=question%3A+Is+Giacomo+Italian%3F+context%3A+Giacomo+is+25+years+old+and+he+was+born+in+Tuscany

In [ ]:
# https://huggingface.co/docs/transformers/model_doc/t5
# https://colab.research.google.com/github/snapthat/TF-T5-text-to-text/blob/master/snapthatT5/notebooks/TF-T5-Datasets%20Training.ipynb